# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902019


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:16,  2.64s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:52,  1.88s/it]

Rendering models:  16%|█▌        | 5/31 [00:05<00:34,  1.34s/it]

Rendering models:  23%|██▎       | 7/31 [00:05<00:23,  1.04it/s]

...

Rendering models:  77%|███████▋  | 24/31 [00:08<00:01,  5.32it/s]

Rendering models:  84%|████████▍ | 26/31 [00:08<00:00,  6.04it/s]

Rendering models:  94%|█████████▎| 29/31 [00:09<00:00,  7.43it/s]

Rendering models: 100%|██████████| 31/31 [00:09<00:00,  7.87it/s]

equidad1                              0.115957
CTidwell3                             0.000347
not-logged-in-60953d21f1d9f596b2fe    0.101472
firejuggler                           0.000349
not-logged-in-21fdfd3434a721014af7    0.000238
not-logged-in-0965bc82e356a9e112d3    0.031363
nickoftona                            0.012964
JessieDawn                            0.000469
tdickey0007                           0.014229
not-logged-in-c016e89f1952c96c30e6    0.000774
dxjdxjhhhh                            0.000370
danielbogomazov                       0.000616
DarioB                                0.003452
Planetme                              0.006723
not-logged-in-20a090b00932d22636ca    0.000132
pangeli5                              0.000765
BraxtonKinney                         0.002609
acapirala                             0.000172
helios1235                            0.000347
ShirleyL                              0.000135
Fla                                   0.000494
Jassels2     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())